In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = ' '.join(text.split())
    return text

In [3]:
def prepareTrainingDfs(dfFake, dfReal):
    for i in range(len(dfFake)):
        news = dfFake.iloc[i, 0]
        news.remove('f')

    for i in range(len(dfReal)):
        news = dfReal.iloc[i, 0]
        news.remove('r')

    words = []
    for i in range(len(dfFake)):
        wordList = dfFake.iloc[i, 0]
        for word in wordList:
            words.append(word)
    dfFake = pd.DataFrame({'word': words})
    dfFake = dfFake.groupby('word').size().reset_index(name='fakeOccurrences')

    words = []
    for i in range(len(dfReal)):
        wordList = dfReal.iloc[i, 0]
        for word in wordList:
            words.append(word)
    dfReal = pd.DataFrame({'word': words})
    dfReal = dfReal.groupby('word').size().reset_index(name='realOccurrences')

    return dfFake, dfReal

In [52]:
dfFake = pd.read_json('../data/train/fakeTrain.json', orient='records', lines=True)
dfReal = pd.read_json('../data/train/realTrain.json', orient='records', lines=True)

In [5]:
def prepararDfsTreino (dfF, dfR):
    # Falsas
    palavrasFalsas = []
    for i in range(len(dfF)):
        noticia = dfF.iloc[i,0][1:]
        palavrasFalsas.extend(noticia)
    dfF = pd.DataFrame({'palavras': palavrasFalsas})
    dfF = dfF.groupby('palavras').size().reset_index(name='aparicoesFalsas')

    palavrasFalsas = []
    for i in range(len(dfR)):
        noticia = dfR.iloc[i,0][1:]
        palavrasFalsas.extend(noticia)
    dfR = pd.DataFrame({'palavras': palavrasFalsas})
    dfR = dfR.groupby('palavras').size().reset_index(name='aparicoesReais')
    palavras = []

    return dfF, dfR

In [53]:
[dfFake, dfReal] = prepararDfsTreino(dfFake, dfReal)

In [91]:
# Junta os dataframes e une as palavras
mergedDf = dfReal.merge(dfFake, how='outer')

In [92]:
# Conta quantas palavras tem em uma e não tem na outra
missingInReal = mergedDf.loc[mergedDf['aparicoesReais'].isna(), 'palavras'].count()
missingInFake = mergedDf.loc[mergedDf['aparicoesFalsas'].isna(), 'palavras'].count()

In [93]:
# Preenche as palavras que não aparecem com 1 pra evitar multiplicação por 0
mergedDf.fillna(1, inplace=True)

In [94]:
# Soma as palavras com mais de zero aparições com um (laplace smoothing)
mergedDf.loc[mergedDf['aparicoesReais'] > 1, 'aparicoesReais'] += 1
mergedDf.loc[mergedDf['aparicoesFalsas'] > 1, 'aparicoesFalsas'] += 1

In [95]:
# Remove linhas onde a coluna 'palavras' é vazia ou só espaço
mergedDf = mergedDf[mergedDf['palavras'].str.strip() != '']

# Mantém só palavras que têm pelo menos uma letra (a–z ou A–Z ou acentuadas)
mergedDf = mergedDf[mergedDf['palavras'].str.match(r'^[A-Za-zÀ-ÖØ-öø-ÿ]+$', na=False)]

In [96]:
# Soma o total de aparições
totalReal = mergedDf['aparicoesReais'].sum()
totalFake = mergedDf['aparicoesFalsas'].sum()

In [97]:
# Balanceia a ordem de grandeza das aparições
mergedDf['aparicoesFalsas'] *= totalReal / totalFake

In [98]:
# Faz o coeficiente de bray curtis
mergedDf['bc'] = abs(mergedDf['aparicoesReais'] - mergedDf['aparicoesFalsas']) / (mergedDf['aparicoesReais'] + mergedDf['aparicoesFalsas'])

In [99]:
# Pega o total de aparições após NLP
totalFilteredFake = mergedDf['aparicoesFalsas'].sum()
totalFilteredReal = mergedDf['aparicoesReais'].sum()

In [100]:
# Faz os calculos das probabilidades
mergedDf['aparicoesFalsas'] = mergedDf['aparicoesFalsas'] / (missingInFake + totalFilteredFake)
mergedDf['aparicoesReais'] = mergedDf['aparicoesReais'] / (missingInReal + totalFilteredReal)

In [ ]:
mergedDf['hellinger'] = ((1/np.sqrt(2)) * (np.sqrt((np.sqrt(mergedDf['aparicoesReais']) - np.sqrt(mergedDf['aparicoesFalsas'])) ** 2)))*10

In [82]:
# Pega o log na base 10
mergedDf['aparicoesFalsas'] = np.log10(mergedDf['aparicoesFalsas'])
mergedDf['aparicoesReais'] = np.log10(mergedDf['aparicoesReais'])

In [ ]:
# Limpa palavras com coeficiente menor que 0.4 (palavras com aparições equivalentes em ambos casos)
# Cria um df pra cada métrica
mergedDf = mergedDf.loc[mergedDf['bc'] > 0.4]

mergedDf = mergedDf.loc[mergedDf['hellinger'] > 0.4]

In [ ]:
# NOTICIA:

cleaned_news = clean_text(news)
words = cleaned_news.split()

# Cria o DataFrame com a lista de palavras limpa
dfNoticia = pd.DataFrame({'palavra': words})

# Junta o df da noticia com o das probabilidades
dfNoticia = dfNoticia.merge(
mergedDf, how='inner', left_on='palavra', right_on='word')

# Corrige os nomes das colunas
iFalsa = dfNoticia['fakeOccurrences'].sum()
iReal = dfNoticia['realOccurrences'].sum()

# Confere o resultado
if (iFalsa > iReal):
    resultado = 'Falsa'
else:
    resultado = 'Real'